In [12]:
# all imported libraries
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import numpy
import pandas as pd
import matplotlib
import csv
import os
import torch
import torchvision
from torchvision import transforms
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline, ResNetForImageClassification, AutoImageProcessor, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric, Image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input, concatenate, Embedding, Conv1D, GlobalMaxPooling1D, TextVectorization
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import PIL
from PIL import Image as im
from PIL import ImageOps as imops
import shutil

In [ ]:
!grep Instagram_Username_Dataset_Updated.csv

^C


####Data to Array

In [2]:
usernames = []
followers = []
following = []
biography = []
profile_picture = []
post_image = []
post_caption = []
labels = []
count = 0

__file__ = "ECS271.ipynb"
script_dir = os.path.dirname(__file__)
script_dir = os.path.join(script_dir, "data")

print("Filling in Usernames")
# fill in all usernames and their label
with open('./Instagram_Username_Dataset_Updated.csv', encoding='utf-8') as fp:
    csvreader = csv.reader(fp)
    for name in csvreader:
        file_path = os.path.join(script_dir, name[0])
        follow_path = os.path.join(file_path, name[0] + "_follower_info.txt")
        if (os.path.isfile(follow_path) == False):
            continue
        usernames.append(name[0])
        labels.append(int(name[1]))

# loop through every user data file=
for name in usernames:
    file_path = os.path.join(script_dir, name)
    follow_path = os.path.join(file_path, name + "_follower_info.txt")
    # add numerical data
    try:
        infile = open(follow_path, "r")
        temp = []
        for line in infile:
            line=line.strip()
            line = int(line)
            temp.append(line)
        infile.close()
    except:
        temp = [0, 0]
    followers.append(int(temp[0]))
    following.append(int(temp[1]))

    # add biography data
    bio_path = os.path.join(file_path, name + "_biography.txt")
    temp = ''
    try:
        infile = open(bio_path, "r")
        for line in infile:
            line = line.strip()
            temp = temp + " " + line
    except:
        temp = ''
    biography.append(temp)
    infile.close()

    # add profile image data
    contain_profile = False
    try:
        for images in os.listdir(file_path):
            if (images.endswith("_profile_pic.jpg")):
                profile_picture.append("./data/" + name + "/" + images)
                contain_profile = True
    except:
        contain_profile = False
    if contain_profile == False:
        profile_picture.append('./data/2018-11-21_19-35-46_UTC_profile_pic.jpg') # load in default pfp if missing

    # add post image data
    post_image_path = os.path.join(file_path, name + "_images")
    # add post image data
    temp = []
    contain_posts = False
    has_four = False
    count = 0
    try:
        for images in os.listdir(post_image_path):
            if (images.endswith(".jpg")):
                temp.append(post_image_path + "/" + images)
                contain_posts = True
                if count == 4:
                    has_four = True
                count += 1
    except:
        contain_posts = False
        temp = []
    if contain_posts == False:
        dst = './data/2018-11-21_19-35-46_UTC_profile_pic.jpg'
        temp.append(dst)
        count += 1
    if has_four == False:
        while count < 4:
            dst = './data/2018-11-21_19-35-46_UTC_profile_pic.jpg'
            temp.append(dst)
            count += 1

    post_image.append(temp)

    # add caption image data
    caption_path = os.path.join(file_path, name + "_captions.txt")
    temp = []
    try:
        infile = open(caption_path, "r")
        for line in infile:
            line = line.strip()
            temp.append(line)
    except:
        temp = []
    post_caption.append(temp)
    infile.close()

Filling in Usernames


In [50]:
# append all to train data
train_data = []
pfp_train_init = []
for x in range (0,len(usernames)):
  temp = [usernames[x], followers[x], following[x], biography[x], profile_picture[x], post_image[x], post_caption[x], labels[x]]
  train_data.append(temp)
  #temp = [profile_picture[x], labels[x]]
  #pfp_train_init.append(temp)

# train_data[x][0] = username
# train_data[x][1] = number of followers
# train_data[x][2] = number of users following
# train_data[x][3] = biography
# train_data[x][4] = string of profile picture
# train_data[x][5] = array of strings of post images
# train_data[x][6] = array of post captions
# train_data[x][7] = binary classification

# convert to df
df = pd.DataFrame(train_data)
#df_pfp = pd.DataFrame(pfp_train_init.append)
df.columns = ["username", "followers", "following", "biography", "profile_picture", "post_images", "post_captions", "label"]
#df_pfp.columns = ["profile_picture","label"]

print(df)


# split training and testing data
x = df.drop("label", axis=1)
y = df["label"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

               username  followers  following  \
0             r.zsyifaa        510        372   
1        shabitanurch31        232        242   
2             tatiazulp         25         31   
3                 00td4        261        316   
4             lariybaby        613        585   
..                  ...        ...        ...   
288  fratelli.valentino       6857         67   
289      hacker_5273747          0          1   
290              1_7482         13          2   
291                3577        107          0   
292             97.5894         18          0   

                                             biography  \
0                                            Bismillah   
1                                                        
2     🎀 | romanticizing life ꒰⁠⑅⁠ᵕ⁠༚⁠ᵕ⁠꒱⁠˖⁠♡ @xsven...   
3                                                        
4                       Asuna🐾 • 𝑒𝓈𝒸𝒶𝓅𝑒 𝓉𝒽𝑒 𝑜𝓇𝒹𝒾𝓃𝒶𝓇𝓎 •   
..                                                 ...   
288  

####Numerical Classification

*   Number of Followers
*   Number of Users Following





In [52]:
# function to process numerical attributes
def process_num_attributes(train, test):
	continuous = ["followers", "following"]
	cs = MinMaxScaler()
	train_cont = cs.fit_transform(train[continuous])
	test_cont = cs.transform(test[continuous])
	return train_cont, test_cont

# apply function on data
x_num_train, x_num_test = process_num_attributes(x_train, x_test)

In [57]:
# mlp created with keras
def create_mlp(dim, regress=False):
    input_layer = Input(shape=(dim,))
    # add dense layers
    layer = Dense(8, activation="relu")(input_layer)
    layer = Dense(4, activation="relu")(layer)
    layer = Dropout(0.5)(layer)
    if regress:
        layer = Dense(1, activation="sigmoid")(layer)
    model = Model(inputs=input_layer, outputs=layer)
    return model

# create/compile model
num_input = Input(shape=(x_num_train.shape[1],))
num_test = Input(shape=(x_num_test.shape[1],))
num_model = create_mlp(2, regress=True)
num_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
num_model.fit(x_num_train, y_train, epochs=10)
loss, accuracy, precision, recall = num_model.evaluate(x_num_test, y_test, verbose=2)
f1 = 2* precision*recall/(precision + recall)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")
print(f"f1-loss: {f1}")

Epoch 1/10
8/8 [==============================] - 1s 2ms/step - loss: 0.7031 - accuracy: 0.5128 - precision_44: 0.5000 - recall_44: 0.2368    
Epoch 2/10
8/8 [==============================] - 0s 2ms/step - loss: 0.6974 - accuracy: 0.5897 - precision_44: 0.5789 - recall_44: 0.5789
Epoch 3/10
8/8 [==============================] - 0s 2ms/step - loss: 0.7047 - accuracy: 0.5299 - precision_44: 0.5714 - recall_44: 0.1404
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 0.6976 - accuracy: 0.5085 - precision_44: 0.4783 - recall_44: 0.0965
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 0.6991 - accuracy: 0.4829 - precision_44: 0.4690 - recall_44: 0.4649
Epoch 6/10
8/8 [==============================] - 0s 4ms/step - loss: 0.6916 - accuracy: 0.6111 - precision_44: 0.5782 - recall_44: 0.7456
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 0.6976 - accuracy: 0.5128 - precision_44: 0.5000 - recall_44: 0.6316
Epoch 8/10
8/8 [=======

####Image Classification:

*   Profile Picture
*   Post Images



In [74]:
#tensorflow pfp_classifier set-up

pfps = []
post_pics = []
lbls = []
for x in range (0,len(usernames)):
    if profile_picture[x] != '':
        pfps.append(profile_picture[x])
        lbls.append(labels[x])
    else:
        print(usernames[x])

    im_temp = []
    for i in range (4):
        if(len(post_image[x]) > i):
            img = im.open(post_image[x][i])
            img = img.resize((512,512), im.LANCZOS)
            im_temp.append(img)

    im12 = im.new('RGB', (im_temp[0].width + im_temp[1].width, im_temp[0].height))
    im34 = im.new('RGB', (im_temp[2].width + im_temp[3].width, im_temp[2].height))
    im12.paste(im_temp[0], (0, 0))
    im12.paste(im_temp[1], (im_temp[0].width, 0))
    im34.paste(im_temp[2], (0, 0))
    im34.paste(im_temp[3], (im_temp[2].width, 0))

    new_image = im.new('RGB', (im12.width, im12.height + im34.height))
    new_image.paste(im12, (0, 0))
    new_image.paste(im34, (0, im12.height))

    new_image = new_image.resize((224,224), im.LANCZOS)
    img_path = script_dir+"/" + usernames[x] + "/joined_img.jpg"
    x_train.loc[x_train.username == usernames[x], "post_images"] = img_path
    x_test.loc[x_test.username == usernames[x], "post_images"] = img_path
    new_image.save(img_path)
    post_pics.append(img_path)

print(pfps)


['./data/r.zsyifaa/2024-04-14_15-08-16_UTC_profile_pic.jpg', './data/shabitanurch31/2024-05-04_13-12-01_UTC_profile_pic.jpg', './data/tatiazulp/2024-02-05_03-39-44_UTC_profile_pic.jpg', './data/00td4/2023-07-30_22-25-56_UTC_profile_pic.jpg', './data/lariybaby/2024-04-22_01-51-40_UTC_profile_pic.jpg', './data/neseekilic/2024-04-25_20-42-22_UTC_profile_pic.jpg', './data/gacha_editsfork/2024-05-01_15-48-31_UTC_profile_pic.jpg', './data/_rawann10_/2024-04-19_11-07-20_UTC_profile_pic.jpg', './data/honeysoymilktea/2024-05-06_22-52-37_UTC_profile_pic.jpg', './data/rarelykind/2024-02-28_21-46-22_UTC_profile_pic.jpg', './data/nsashleyx/2024-05-11_23-44-28_UTC_profile_pic.jpg', './data/ms_grayned/2022-05-29_20-30-31_UTC_profile_pic.jpg', './data/ilovefashion719/2024-05-01_18-27-54_UTC_profile_pic.jpg', './data/5star.iha/2018-11-21_19-35-46_UTC_profile_pic.jpg', './data/ranniiyyaa/2024-03-29_14-32-57_UTC_profile_pic.jpg', './data/kevon.365/2023-06-25_03-56-54_UTC_profile_pic.jpg', './data/hennbro

In [86]:
#tensorflow pfp_classifier set-up p2
import tensorflow as tf

# Define a function to load and preprocess images
def load_and_preprocess_image(path):
    # Read the image file
    print(path["path"])
    image = tf.io.read_file(path["path"])
    # Decode the image
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize the image to a fixed size (e.g., 224x224)
    image = tf.image.resize(image, [224, 224])
    # Normalize the image to [0, 1] range
    image = image / 255.0
    return dict(image = image)
# Apply the function to the dataset
dataset = dict(path = x_train["profile_picture"].tolist(), label = y_train)
post_dataset = dict(path = x_train["post_images"].tolist(), label = y_train)
dataset_test = dict(path = x_test["profile_picture"].tolist(), label = y_test)
post_dataset_test = dict(path = x_test["post_images"].tolist(), label = y_test)

dataset = Dataset.from_dict(dataset)
post_dataset = Dataset.from_dict(post_dataset)
dataset_test = Dataset.from_dict(dataset_test)
post_dataset_test = Dataset.from_dict(post_dataset_test)


image_train = dataset.map(load_and_preprocess_image)["image"]
post_train = post_dataset.map(load_and_preprocess_image)["image"]
image_test = dataset_test.map(load_and_preprocess_image)["image"]
post_test = post_dataset_test.map(load_and_preprocess_image)["image"]

#dataset_size = len(pfps)
#train_size = int(0.8 * dataset_size)  # 80% for training
#val_size = dataset_size - train_size  # Remaining 20% for validation




#image_train, image_test, image_label_train, image_label_test = train_test_split(dataset["image"], dataset["label"], test_size=0.2, random_state=42)
#post_train, post_test, post_label_train, post_label_test = train_test_split(post_dataset["image"], post_dataset["label"], test_size=0.2, random_state=42)
# Split the dataset
#train_dataset = dataset.take(train_size)
#val_dataset = dataset.skip(train_size)

# Shuffle, batch, and prefetch the datasets
#train_dataset = train_dataset.shuffle(buffer_size=1000).batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#val_dataset = val_dataset.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

./data/inovationunlimited/2024-06-02_15-06-02_UTC_profile_pic.jpg
./data/_naidu._.fjx_/2024-04-11_10-58-24_UTC_profile_pic.jpg
./data/tbaum17/2015-08-28_05-25-29_UTC_profile_pic.jpg
./data/trin.spadaro/2024-05-15_16-05-03_UTC_profile_pic.jpg
./data/hayeyashi/2019-10-22_00-41-50_UTC_profile_pic.jpg
./data/likeeebutter/2018-11-21_19-35-46_UTC_profile_pic.jpg
./data/ayang_angga84/2023-11-03_03-46-55_UTC_profile_pic.jpg
./data/ashley.laroc/2024-03-11_00-57-24_UTC_profile_pic.jpg
./data/rebeccasugar/2023-09-06_01-30-03_UTC_profile_pic.jpg
./data/lover.feedzzz/2018-11-21_19-35-46_UTC_profile_pic.jpg
./data/wwwww20142020/2021-09-18_00-05-27_UTC_profile_pic.jpg
./data/kevon.365/2023-06-25_03-56-54_UTC_profile_pic.jpg
./data/nsashleyx/2024-05-11_23-44-28_UTC_profile_pic.jpg
./data/itschuzzy/2024-04-04_02-48-08_UTC_profile_pic.jpg
./data/emorie.m/2024-05-03_23-31-35_UTC_profile_pic.jpg
./data/xxmdmd9/2018-11-21_19-35-46_UTC_profile_pic.jpg
./data/not_them/2018-12-20_20-19-16_UTC_profile_pic.jpg


Map:   0%|          | 0/234 [00:00<?, ? examples/s]

data/haqwyh/joined_img.jpg
data/naiduveenal/joined_img.jpg
data/khvan3934/joined_img.jpg
data/alejo._fajardo/joined_img.jpg
data/hayeyashi/joined_img.jpg
data/lyingisland/joined_img.jpg
data/peju.ang4080/joined_img.jpg
data/trin.spadaro/joined_img.jpg
data/rebeccasugar/joined_img.jpg
data/follow_tanveer.gilll/joined_img.jpg
data/cdppdc6300/joined_img.jpg
data/kevon.365/joined_img.jpg
data/nsashleyx/joined_img.jpg
data/richardromanashton/joined_img.jpg
data/jenna_segal/joined_img.jpg
data/inovationunlimited/joined_img.jpg
data/not_them/joined_img.jpg
data/hennbro1/joined_img.jpg
data/1_7482/joined_img.jpg
data/liza_lyz245/joined_img.jpg
data/toms_tits_and_boobies/joined_img.jpg
data/emorie.m/joined_img.jpg
data/alex.aa4/joined_img.jpg
data/aussy_bakkes/joined_img.jpg
data/action_ou_veriter_hot/joined_img.jpg
data/likeeebutter/joined_img.jpg
data/xxmdmd9/joined_img.jpg
data/jokersson7/joined_img.jpg
data/verra_uliani/joined_img.jpg
data/coach.kevin/joined_img.jpg
data/jasminepak/joined_i

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

./data/shivam10_7/2023-01-01_20-14-20_UTC_profile_pic.jpg
./data/coach.kevin/2024-05-21_18-36-26_UTC_profile_pic.jpg
./data/macsadbh/2022-11-05_12-38-09_UTC_profile_pic.jpg
./data/xusein_xusein/2018-11-21_19-35-46_UTC_profile_pic.jpg
./data/georgepeters969/2023-11-06_06-51-41_UTC_profile_pic.jpg
./data/acjaiajsksl/2018-11-21_19-35-46_UTC_profile_pic.jpg
./data/khvan3934/2024-03-12_15-50-06_UTC_profile_pic.jpg
./data/big.nng/2018-11-21_19-35-46_UTC_profile_pic.jpg
./data/rarelykind/2024-02-28_21-46-22_UTC_profile_pic.jpg
./data/mattwener/2024-02-06_03-35-06_UTC_profile_pic.jpg
./data/ritvakorhonen2023/2023-10-25_08-42-37_UTC_profile_pic.jpg
./data/cassiopeia_080/2023-05-13_08-30-42_UTC_profile_pic.jpg
./data/zled.art/2023-11-13_02-46-49_UTC_profile_pic.jpg
./data/_bex_ruz_0924/2018-11-21_19-35-46_UTC_profile_pic.jpg
./data/kingvaleri/2017-05-26_07-09-39_UTC_profile_pic.jpg
./data/kalanididit/2018-12-13_07-27-55_UTC_profile_pic.jpg
./data/kaitlynjb2006_/2023-05-13_12-07-53_UTC_profile_pi

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

data/jabonthegringo/joined_img.jpg
data/safeway_/joined_img.jpg
data/macsadbh/joined_img.jpg
data/jdjdbxjxjbxxh/joined_img.jpg
data/onlyn351/joined_img.jpg
data/his.mayanti/joined_img.jpg
data/crous.dale/joined_img.jpg
data/ch.ristian5363/joined_img.jpg
data/rarelykind/joined_img.jpg
data/azizo_ahmed_/joined_img.jpg
data/ayang_angga84/joined_img.jpg
data/betwbeth/joined_img.jpg
data/zled.art/joined_img.jpg
data/_.loret0/joined_img.jpg
data/a_ii9i4/joined_img.jpg
data/beeladykim/joined_img.jpg
data/kaitlynjb2006_/joined_img.jpg
data/neseekilic/joined_img.jpg
data/fgott_ph/joined_img.jpg
data/trixicen/joined_img.jpg
data/acjaiajsksl/joined_img.jpg
data/maestrokirbs/joined_img.jpg
data/bex092/joined_img.jpg
data/milfs.indian/joined_img.jpg
data/grillospickles/joined_img.jpg
data/jenna0/joined_img.jpg
data/cassiopeia_080/joined_img.jpg
data/23.1osohee/joined_img.jpg
data/gaia_filia/joined_img.jpg
data/najurdx/joined_img.jpg
data/curtwilde/joined_img.jpg
data/phia.henry/joined_img.jpg
data/

In [91]:
print(x_train)

             username  followers  following  \
210            haqwyh       1283        359   
229       naiduveenal        395        701   
226         khvan3934         44        179   
73     alejo._fajardo        325        292   
25          hayeyashi        699        856   
..                ...        ...        ...   
188  trevordiaz_music        114        168   
71       ashley.laroc        516        548   
106           haruy.r          1          1   
270    cock_girl_9900          2          0   
102     hot_dilf_feet          0          0   

                                             biography  \
210                                                      
229   . . . ❤️‍🩹❤️‍🩹 . . . Main account @_naidu._.f...   
226                                         Yurikh1998   
73    20| 🇬🇹 Kean ‘27 📚Adare (2023) 📚Julian (2021) ...   
25    Just a Silly Little Clown that doesn't even k...   
..                                                 ...   
188   🖥 Computer/Electrical E

In [87]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

#RESNET-50 model pre-trained and fine-tunable
image_inputs = Input(shape=(224,224,3))
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-10:]:  # Adjust the number of layers to unfreeze as needed
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Example of a fully connected layer
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)  # Change `num_classes` to your number of classes

image_train = numpy.asarray(image_train)
image_test = numpy.asarray(image_test)
post_train = numpy.asarray(post_train)
post_test = numpy.asarray(post_test)



pfp_model = Model(inputs=base_model.input, outputs=predictions)
pfp_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
pfp_model.fit(image_train, y_train, epochs=10, verbose = 0)


post_model = Model(inputs=base_model.input, outputs=predictions)
post_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
post_model.fit(post_train, y_train, epochs=10, verbose = 0)

loss, accuracy, precision, recall = pfp_model.evaluate(image_test, y_test, verbose=2)
f1 = 2* precision*recall/(precision + recall)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")
print(f"f1-loss: {f1}")

loss, accuracy, precision, recall = post_model.evaluate(post_test, y_test, verbose=2)
f1 = 2* precision*recall/(precision + recall)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")
print(f"f1-loss: {f1}")


2/2 - 7s - loss: 1.0416 - accuracy: 0.7119 - precision_48: 0.7895 - recall_48: 0.5357 - 7s/epoch - 3s/step
Test Loss: 1.0415918827056885
Test Accuracy: 0.7118644118309021
f1-loss: 0.6382978694802653
2/2 - 7s - loss: 0.6979 - accuracy: 0.8305 - precision_49: 0.8462 - recall_49: 0.7857 - 7s/epoch - 4s/step
Test Loss: 0.697911262512207
Test Accuracy: 0.8305084705352783
f1-loss: 0.8148148099090825


####Text Classification:

*   Usernames
*   Biography
*   Post Captions



In [99]:
#ADD FOLLOWING:
#Edit tokenization
#Create train, and dev set
def list_to_dict(data, labeled):
    sentences1 = []
    sentences2 = []
    sentences3 = []
    labels = []
    for item in data:
        sentences1.append(item[0])
        sentences2.append(item[3])
        sentences3.append(item[6])
        if labeled:
            labels.append(item[7])
    if labeled:
        convert_dict = dict(sentence1 = sentences1, sentence2 = sentences2, sentence3 = sentences3, label = labels)
    else:
        convert_dict = dict(sentence1 = sentences1, sentence2 = sentences2, sentences3 = sentences3)
    return convert_dict


#Tokenizes the dataset, by creating the input_id from tokenizing the two sentences
def tokenize_function(dataset):
    captionlist = []
    for index, row in dataset.iterrows():
        tempcaptions = ""
        for caption in row["post_captions"]:
            tempcaptions += caption
        user_caption = row["username"] + "[SEP]" + row["biography"] + "[SEP]" + tempcaptions
        captionlist.append(user_caption)
    #Change to usernames [0], biography [3], post captions [6]
    return captionlist
    return dict(tokenizer(dataset["sentence1"], dataset["sentence2"], captionlist, truncation=True, padding=True, return_tensors="np"))


max_features = 200000
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
    max_tokens = max_features,
    output_mode = "int",
    output_sequence_length = sequence_length
)

text_train = x_train
text_val = x_test



#text_train = text_train.map(vectorize_text)

text_inputs = Input(shape=(1,), dtype="string")
x = vectorize_layer(text_inputs)
x = Embedding(max_features, embedding_dim)(x)
x = Dropout(0.5)(x)
x = Conv1D(128, 7, padding = "valid", activation = "relu", strides = 3) (x)
x = Conv1D(128, 7, padding = "valid", activation = "relu", strides = 3) (x)
x = GlobalMaxPooling1D() (x)
x = Dense(128, activation = "relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation = "sigmoid", name = "outputs")(x)
text_model = Model(text_inputs, outputs)
#Finetune RoBERTa large with the training data
#text_model = TFAutoModelForSequenceClassification.from_pretrained("FacebookAI/roberta-large", num_labels=2)
#tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

#Turn training, dev, and test set into datasets
#Tokenize the datasets for processing
tokenized_train_dataset = pd.DataFrame(data = tokenize_function(text_train))
tokenized_train_dataset.col = ["text"]
tokenized_val_dataset = pd.DataFrame(data = tokenize_function(text_val))
tokenized_val_dataset.col = ["text"]
vectorize_layer.adapt(tokenized_train_dataset)
#tokenized_dev_dataset = dev_dataset.map(tokenize_function, batched=True)

text_model.compile(optimizer="adam", metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()], loss = "binary_crossentropy")  # No loss argument!

text_model.fit([tokenized_train_dataset], y_train, epochs=10)
loss, accuracy, precision, recall = text_model.evaluate(tokenized_val_dataset, y_test, verbose=2)
f1 = 2* precision*recall/(precision + recall)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")
print(f"f1-loss: {f1}")

C:\Users\Aaron Nguyen\AppData\Local\Temp\ipykernel_19592\688156362.py:71: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  tokenized_train_dataset.col = ["text"]
C:\Users\Aaron Nguyen\AppData\Local\Temp\ipykernel_19592\688156362.py:73: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  tokenized_val_dataset.col = ["text"]


Epoch 1/10
8/8 [==============================] - 4s 228ms/step - loss: 0.6533 - accuracy: 0.7009 - precision_54: 0.7075 - recall_54: 0.6579
Epoch 2/10
8/8 [==============================] - 2s 230ms/step - loss: 0.5234 - accuracy: 0.8376 - precision_54: 0.7794 - recall_54: 0.9298
Epoch 3/10
8/8 [==============================] - 2s 229ms/step - loss: 0.3595 - accuracy: 0.8632 - precision_54: 0.8154 - recall_54: 0.9298
Epoch 4/10
8/8 [==============================] - 2s 228ms/step - loss: 0.3202 - accuracy: 0.8803 - precision_54: 0.8413 - recall_54: 0.9298
Epoch 5/10
8/8 [==============================] - 2s 230ms/step - loss: 0.2724 - accuracy: 0.9060 - precision_54: 0.8538 - recall_54: 0.9737
Epoch 6/10
8/8 [==============================] - 2s 226ms/step - loss: 0.2197 - accuracy: 0.9231 - precision_54: 0.8636 - recall_54: 1.0000
Epoch 7/10
8/8 [==============================] - 2s 230ms/step - loss: 0.2075 - accuracy: 0.9316 - precision_54: 0.8828 - recall_54: 0.9912
Epoch 8/10
8/

####Multimodal Neural Network Model

Combine all features and feed through single model



In [22]:
post_inputs = Input(shape=(224,224,3))
# concatenate all separate inputs and process
combined = concatenate([num_model(num_input), text_model(text_inputs), pfp_model(image_inputs), post_model(post_inputs)])
z = Dense(10, activation="relu")(combined)
z = Dense(1, activation="sigmoid")(z)

# final model
model = Model(inputs=[num_input, text_inputs, image_inputs, post_inputs], outputs=z)

# training
model.compile(loss="binary_crossentropy", metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.fit([x_num_train, tokenized_train_dataset, image_train, post_train], y_train, epochs=20, batch_size=8, verbose=2)

# evaluate based on loss and accuracy
loss, accuracy, precision, recall = model.evaluate([x_num_test, tokenized_val_dataset, image_test, post_test], y_test, verbose=2)
f1 = 2* precision*recall/(precision + recall)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")
print(f"f1-loss: {f1}")

Epoch 1/20
17/17 - 33s - loss: 0.6521 - accuracy: 0.5263 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - 33s/epoch - 2s/step
Epoch 2/20
17/17 - 10s - loss: 0.6320 - accuracy: 0.5263 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - 10s/epoch - 587ms/step
Epoch 3/20
17/17 - 10s - loss: 0.6200 - accuracy: 0.5263 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00 - 10s/epoch - 590ms/step
Epoch 4/20
17/17 - 11s - loss: 0.6133 - accuracy: 0.6316 - precision_5: 1.0000 - recall_5: 0.2222 - 11s/epoch - 622ms/step
Epoch 5/20
17/17 - 10s - loss: 0.6069 - accuracy: 0.9549 - precision_5: 0.9130 - recall_5: 1.0000 - 10s/epoch - 614ms/step
Epoch 6/20
17/17 - 11s - loss: 0.6010 - accuracy: 0.9549 - precision_5: 0.9130 - recall_5: 1.0000 - 11s/epoch - 638ms/step
Epoch 7/20
17/17 - 11s - loss: 0.5944 - accuracy: 0.9549 - precision_5: 0.9130 - recall_5: 1.0000 - 11s/epoch - 633ms/step
Epoch 8/20
17/17 - 11s - loss: 0.5881 - accuracy: 0.9549 - precision_5: 0.9130 - recall_5: 1.0000 - 11s/epoch - 633ms/

In [ ]:
# create prediction from model
y_pred = model.predict([x_num_test, x_text_test])
y_pred_classes = (y_pred > 0.5).astype("int32")  # Convert probabilities to binary class predictions

# print report
print(classification_report(y_test, y_pred_classes))